## Goal
In this notebook, I will clean and preprocess the data, then feed it to some basic machine learning models, with a goal of predicting the number of days until a given university acts (relative to the first university's action in that category). So, if I put the features corresponding to WashU in my model, I may get 3 as an output, which would mean WashU is predicted to impose covid restrictions 3 days after the first mover.

**Note (important):** There are many problems in my input data. First, there's not enough of it (only 51 samples) due to the difficulty of collecting the data. Also, there's likely not enough features--a large one I'm missing is the incidence rate of COVID in each zip code during the period where decisions were made. This may make the model more accurate. Second, there are a lot of interconnectivities between the data, meaning the iid assumption is violated. It's almost always realistic for this assumption to be violated, but in this model it is especially problematic, as universities likely base their decisions on the status of other, similar universities. So, this model is flawed. However, it is based on a novel problem I thought about and could possibly possess some small degree of predictive power. Whatever the outcome, it will have been good practice, especially with getting the data into a workable form.

Notes for running:
- Can change ```census_vars``` to include any variables from the Census Bureau's ACS 5-year data

In [577]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Preparing the Data

In [578]:
covid_dates = pd.read_excel("cleaned_university_covid_dates.xlsx")

In [579]:
covid_dates.head()

,displayName,Unofficial Ranking,rankingDisplayRank,state,city,zip,Ivy,institution type,Date of Spring 2020 Move Online (first action to move classes online and tell students not to return to campus (some require it); many acted later to extend move to end of semester;),Date of Vaccine Requirement for Students in FL2021,Date of Booster Requirement,Date of Spring 2022 Move Online/Delay,description
0,Princeton University,1,#1,NJ,Princeton,8544,True,Private,2020-03-11,2021-04-20,2021-12-16,2021-12-27,The ivy-covered campus of Princeton University...
1,Columbia University,2,#2,NY,New York,10027,True,Private,2020-03-12,2021-04-19,2021-12-16,2021-12-22,Columbia University has three undergraduate sc...
2,Harvard University,3,#2,MA,Cambridge,2138,True,Private,2020-03-10,2021-05-05,2021-12-16,NaT,Harvard University is a private institution in...
3,Massachusetts Institute of Technology,4,#2,MA,Cambridge,2139,False,Private,2020-03-10,2021-04-30,2021-12-13,NaT,Though the Massachusetts Institute of Technolo...
4,Yale University,5,#5,CT,New Haven,6520,True,Private,2020-03-10,2021-04-19,2021-12-17,2021-12-22,"Yale University, located in New Haven, Connect..."


First, I'll change the titles of the columns; they are a bit too long. Also, I'm not using the descriptions from USNews so I can drop that, as well as the display rank and display name and city (we'll just use state and zip here)

In [580]:
covid_dates = covid_dates.rename(columns=
                       {"Date of Spring 2020 Move Online (first action to move classes online and tell students not to return to campus (some require it); many acted later to extend move to end of semester;)": "Spring2020",
                        "Date of Vaccine Requirement for Students in FL2021": "FirstVaccine",
                        "Date of Booster Requirement": "Booster",
                        "Date of Spring 2022 Move Online/Delay": "Spring2022"}).drop(columns=["description", "displayName", "rankingDisplayRank", "city"])

In [581]:
covid_dates.head()

,Unofficial Ranking,state,zip,Ivy,institution type,Spring2020,FirstVaccine,Booster,Spring2022
0,1,NJ,8544,True,Private,2020-03-11,2021-04-20,2021-12-16,2021-12-27
1,2,NY,10027,True,Private,2020-03-12,2021-04-19,2021-12-16,2021-12-22
2,3,MA,2138,True,Private,2020-03-10,2021-05-05,2021-12-16,NaT
3,4,MA,2139,False,Private,2020-03-10,2021-04-30,2021-12-13,NaT
4,5,CT,6520,True,Private,2020-03-10,2021-04-19,2021-12-17,2021-12-22


Now, instead of using dates, I will use days after the first university acted in each category.

In [582]:
covid_dates_only_d = covid_dates[['Spring2020', 'FirstVaccine', 'Booster', 'Spring2022']]
first_dates = covid_dates_only_d.min()
date_diff = covid_dates_only_d - first_dates
covid_dates[['Spring2020', 'FirstVaccine', 'Booster', 'Spring2022']] = date_diff.apply(lambda x: x.dt.days)
covid_dates.head()

,Unofficial Ranking,state,zip,Ivy,institution type,Spring2020,FirstVaccine,Booster,Spring2022
0,1,NJ,8544,True,Private,5,18.0,10.0,11.0
1,2,NY,10027,True,Private,6,17.0,10.0,6.0
2,3,MA,2138,True,Private,4,33.0,10.0,NaN
3,4,MA,2139,False,Private,4,28.0,7.0,NaN
4,5,CT,6520,True,Private,4,17.0,11.0,6.0


There are two different types of dates here: those that cause a shift to online learning and those that mandate vaccinations. I will average the days for both these categories and create a new column signifying 0 for the former type of date (move online) and 1 for the latter (vaccination).

Note that there are some null values in the data, which signify if a university did not act on that specific outcome. This is problematic as if we ignore/impute them, then we lose valuable data--perhaps a certain combination of features entices schools not to act. This would then be ignored in our data (see the section [exploring a different task](#exploring-a-different-task) below for more). So, I will consider not acting as a university taking the largest possible number of days to make the decision * 2 (an arbitrary choice). Thus the predicted outcome will be much higher if a university did not act. I will impute the values below.

In [583]:
arb_constant = 2
max_dates_online = covid_dates[['Spring2020', 'FirstVaccine', 'Booster', 'Spring2022']].max() * arb_constant
covid_dates_imputed = covid_dates.fillna(max_dates_online)

To ignore the above (no imputing), comment out the above.

In [584]:
# covid_dates_imputed = covid_dates

In [585]:
covid_dates_imputed['online'] = covid_dates_imputed[['Spring2020', 'Spring2022']].mean(axis=1).dropna()
covid_dates_imputed['vaccine'] = covid_dates_imputed[['FirstVaccine', 'Booster']].mean(axis=1).dropna()
covid_dates_cleaned = covid_dates_imputed.drop(columns=['Spring2020', 'FirstVaccine', 'Booster', 'Spring2022'])

Now, I'll separate each university into two rows, one for online and one for vaccine.

In [586]:
covid_dates_cleaned.rename(columns={'Unofficial Ranking': 'ranking', 'Ivy': 'ivy', 'institution type': 'institution_type'}, 
                           inplace=True)

In [587]:
covid_dates_cleaned = covid_dates_cleaned.melt(id_vars=['ranking', 'state', 'zip', 'ivy', 'institution_type'], var_name="decision_type", value_name="days_after_first")

Finally, I'll change ranking to be in groups of 10, i.e., rankings 1-10 encoded with 0, rankings 11-20 encoded with 1, etc (note that 51 will be encoded with a 4). This makes more sense as individual rankings don't matter as much as general cases.

In [588]:
covid_dates_cleaned['ranking'] = (covid_dates_cleaned['ranking'] - 1)//10
covid_dates_cleaned['ranking'] = covid_dates_cleaned['ranking'].where(covid_dates_cleaned["ranking"] != 5, 4)

In [589]:
covid_dates_cleaned.head()

,ranking,state,zip,ivy,institution_type,decision_type,days_after_first
0,0,NJ,8544,True,Private,online,8.0
1,0,NY,10027,True,Private,online,6.0
2,0,MA,2138,True,Private,online,24.0
3,0,MA,2139,False,Private,online,24.0
4,0,CT,6520,True,Private,online,5.0


### Extracting Data from Zip Codes Using the Census Bureau Data API
As inspired by [this article on leveraging value from postal codes](https://towardsdatascience.com/leveraging-value-from-postal-codes-naics-codes-area-codes-and-other-funky-arse-categorical-be9ce75b6d5a), I'll extract mean information for each zip code and use as features, not the zip code itself. I'm following [these reddit comments](https://www.reddit.com/r/datasets/comments/i2g55u/demographic_data_sets_by_zip_code_free/) as a guide. I will be using the [census package](https://pypi.org/project/census/) for Python and the census api. Also, I'm storing my api key as an env variable so I don't accidentally push it to Github. See [here](https://able.bio/rhett/how-to-set-and-get-environment-variables-in-python--274rgt5) for more.

"This product uses the Census Bureau Data API but is not endorsed or certified by the Census Bureau."

In [590]:
from census import Census
import os
census_api_key = os.getenv('api_key_census')
c = Census(census_api_key, year=2020)

I will get data from [ACS 5-year data](https://www.census.gov/data/developers/data-sets/acs-5year.html) in 2020, as that's the closest available year to the pandemic and will give us a general sense of the demographic variables of the location the university is in. I would prefer to use data from 2020-2022, which is when colleges made these decisions, but unfortunately that's not available. There is ACS 1-year data, but that doesn't have zip code support, at least not in the package.

Here's a list of variables.

In [591]:
pd.DataFrame.from_dict(c.acs5.tables())

,name,description,variables,universe
0,B17015,POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILI...,http://api.census.gov/data/2020/acs/acs5/group...,FAMILY
1,B18104,SEX BY AGE BY COGNITIVE DIFFICULTY,http://api.census.gov/data/2020/acs/acs5/group...,NONINST_05_OVER
2,B17016,POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILI...,http://api.census.gov/data/2020/acs/acs5/group...,FAMILY
3,B18105,SEX BY AGE BY AMBULATORY DIFFICULTY,http://api.census.gov/data/2020/acs/acs5/group...,NONINST_05_OVER
4,B17017,POVERTY STATUS IN THE PAST 12 MONTHS BY HOUSEH...,http://api.census.gov/data/2020/acs/acs5/group...,HSHLD
...,...,...,...,...
1135,B99131,ALLOCATION OF MARITAL STATUS FOR FEMALES 15 TO...,http://api.census.gov/data/2020/acs/acs5/group...,WOMEN_15_50
1136,B09018,RELATIONSHIP TO HOUSEHOLDER FOR CHILDREN UNDER...,http://api.census.gov/data/2020/acs/acs5/group...,POP_18_UNDER_HSHLD_EXCL
1137,B09019,HOUSEHOLD TYPE (INCLUDING LIVING ALONE) BY REL...,http://api.census.gov/data/2020/acs/acs5/group...,TOTAL_POP
1138,B99132,ALLOCATION OF FERTILITY OF WOMEN 15 TO 50 YEARS,http://api.census.gov/data/2020/acs/acs5/group...,WOMEN_15_50


In this project, I will just get an assortment of basic demographic variables into a new dataframe, then merge it with my university data. A more sophisticated analysis could have a greater basis for choosing such variables.

Save variables to excel for easy browsing. Use [this guide to subject definitions](https://www2.census.gov/programs-surveys/acs/tech_docs/subject_definitions/2020_ACSSubjectDefinitions.pdf) for detailed help.

In [592]:
# pd.DataFrame.from_dict(c.acs5.tables()).to_excel('acs5_vars.xlsx')

An example for median income in past 12 months for Columbia's zip code.

In [593]:
c.acs5.state_zipcode('B07011_001E', 36, 10027)

[{'B07011_001E': 32100.0, 'zip code tabulation area': '10027'}]

I need to collect the codes for all the variables and the fips for each state (using [unitedstates package](https://github.com/unitedstates/python-us)). First I'll do fips.

In [594]:
import us

In [595]:
state_fips = us.states.mapping('abbr', 'fips')
covid_dates_cleaned['state_fips'] = covid_dates_cleaned['state'].apply(lambda x: state_fips[x])
covid_dates_cleaned.head()

,ranking,state,zip,ivy,institution_type,decision_type,days_after_first,state_fips
0,0,NJ,8544,True,Private,online,8.0,34
1,0,NY,10027,True,Private,online,6.0,36
2,0,MA,2138,True,Private,online,24.0,25
3,0,MA,2139,False,Private,online,24.0,25
4,0,CT,6520,True,Private,online,5.0,09


Now, I need to **select variables**. I think median income, population size, and political leaning could be interesting variables to start with. Note that political leaning is not available; I need to obtain it myself. It's specifically important here because Republicans and Democrats have different COVID responses; for example, a lot of Republican-led states may have more lax restrictions.

In [597]:
census_vars = {'B07011_001E': 'median_income', 'B01003_001E': 'total_population'}

Note that some zip codes don't have corresponding values. See the zip code for Princeton, NJ.

In [599]:
v = list(census_vars.keys())[0]

In [600]:
c.acs5.state_zipcode(v, 34, '08544')

[]

So, I'll try to get the corresponding variables from one geographic level up. For example, the county fips for Mercer County (where Princeton, NJ is located) is 021. Using this gets a result.

In [601]:
c.acs5.state_county(v, 34, '021')

[{'B07011_001E': 37223.0, 'state': '34', 'county': '021'}]

So, I need to extract the county fips from the zip code. I can do this using the [US Zipcodes to County State to FIPS Crosswalk](https://www.kaggle.com/datasets/danofer/zipcodes-county-fips-crosswalk) dataset. Note that the first two digits of the county fips correspond to the state fips, and the last three digits of STCOUNTYFP correspond to the county fips [as seen here](https://www2.census.gov/geo/pdfs/maps-data/data/tiger/tiger2006se/app_a03.pdf).

In [602]:
county_zips = pd.read_csv('zip-county-fips/ZIP-COUNTY-FIPS_2017-06.csv')
county_zips.head()

,ZIP,COUNTYNAME,STATE,STCOUNTYFP,CLASSFP
0,36003,Autauga County,AL,1001,H1
1,36006,Autauga County,AL,1001,H1
2,36067,Autauga County,AL,1001,H1
3,36066,Autauga County,AL,1001,H1
4,36703,Autauga County,AL,1001,H1


In [603]:
covid_dates_cleaned = covid_dates_cleaned.merge(county_zips[["ZIP", "STATE", "STCOUNTYFP"]], left_on=["state", "zip"], right_on=["STATE", "ZIP"]).drop(columns=["ZIP", "STATE"])

In [604]:
covid_dates_cleaned["county_fips"] = covid_dates_cleaned["STCOUNTYFP"]%1000

Now, try through the api with county

In [605]:
c.acs5.state_county(v, 34, '021')

[{'B07011_001E': 37223.0, 'state': '34', 'county': '021'}]

In [606]:
covid_dates_cleaned[['state_fips', 'county_fips']]

,state_fips,county_fips
0,34,21
1,34,21
2,36,61
3,36,61
4,25,17
...,...,...
109,42,95
110,25,25
111,25,25
112,39,49


In [607]:
c.acs5.state_county(v, 34, '021')

[{'B07011_001E': 37223.0, 'state': '34', 'county': '021'}]

This works, so I will use counties, which should all have values (unlike zip codes). Before I call the api, I need to make sure I found county fips for all counties present.

In [608]:
covid_dates_cleaned['county_fips'].isna().sum()

0

Also, I need to pad zeros for the api.

In [609]:
covid_dates_cleaned['county_fips_str'] = covid_dates_cleaned['county_fips'].astype(str).str.zfill(3)

Now, create a function that we can apply to each row of the dataframe that will call the api and get the desired census variables (held in ```census_vars```). Note that we only want this for unique rows.

In [610]:
api_return_cols = list(census_vars.values()) + ['state', 'county']
def separate_county_fips(x, *v):    
    api_return = c.acs5.state_county(v, x[0], x[1]) # returns dict in a list if found; empty list if not    
    try:
        api_return_clean = pd.Series(api_return[0])
        return api_return_clean.rename(census_vars) 
    except:
        no_api_return = pd.Series(index=api_return_cols, dtype='object')
        no_api_return[['state', 'county']] = x.values
        return no_api_return

Note that I can obtain multiple fields from the same api call. This will be better than using a for-loop.

In [611]:
census_var_names = list(census_vars.keys())
census_var_names

['B07011_001E', 'B01003_001E']

In [613]:
c.acs5.state_county(census_var_names, 34, '021')

[{'B07011_001E': 37223.0,
  'B01003_001E': 368085.0,
  'state': '34',
  'county': '021'}]

Do a quick check to make sure it handles non-existant inputs correctly.

In [614]:
testing_df = pd.DataFrame({'state_fips': [34, 36], 'county_fips_str': ['111', '061']})
testing_output = testing_df.drop_duplicates().apply(separate_county_fips, args=(census_var_names), axis=1)
testing_output

,median_income,total_population,state,county
0,NaN,NaN,34,111
1,52409.0,1629153.0,36,061


Now, I need to call the api on all my census variables using the above. Create a new df with target values, then merge.

In [615]:
census_vars_counties = covid_dates_cleaned[['state_fips', 'county_fips_str']].drop_duplicates().apply(separate_county_fips, args=(census_var_names), axis=1)
census_vars_counties.head()

,median_income,total_population,state,county
0,37223.0,368085.0,34,021
2,52409.0,1629153.0,36,061
4,48230.0,1605899.0,25,017
8,36670.0,855733.0,09,009
10,50030.0,1924379.0,06,085


In [625]:
covid_dates_cleaned = (covid_dates_cleaned.merge(census_vars_counties, 
                                                 left_on=['state_fips', 'county_fips_str'], 
                                                 right_on=['state', 'county'], 
                                                 suffixes=('', '_new'))
                       .drop(columns=['state_new', 'county']))

covid_dates_cleaned.head()

,ranking,state,zip,ivy,institution_type,decision_type,days_after_first,state_fips,STCOUNTYFP,county_fips,county_fips_str,median_income,total_population
0,0,NJ,8544,True,Private,online,8.0,34,34021,21,021,37223.0,368085.0
1,0,NJ,8544,True,Private,vaccine,14.0,34,34021,21,021,37223.0,368085.0
2,0,NY,10027,True,Private,online,6.0,36,36061,61,061,52409.0,1629153.0
3,0,NY,10027,True,Private,vaccine,13.5,36,36061,61,061,52409.0,1629153.0
4,2,NY,10012,False,Private,online,23.5,36,36061,61,061,52409.0,1629153.0


### Political Leaning
There is no simple way to gauge political leanings, so I will use composition of state legislature as a proxy. A college town may often lean left, but the state legislature represents overall political sentiment and has a direct impact on COVID guidelines, so I think it'll be more useful. I will use the data from [the ncsl](https://www.ncsl.org/Portals/1/Documents/Elections/Legis_Control_2020_April%201.pdf) for who controlled the state legislature in 2020, filled in as Rep for Nebraska after a Google and Dem for DC as it votes heavily Democratic. Hardcoded because it's only 50 samples and pretty basic.

In [116]:
political_control_state = {'DC': 'Dem', 'AL': 'Rep', 'AK': 'Rep', 'AZ': 'Rep', 'AR': 'Rep', 'CA': 'Dem', 'CO': 'Dem', 'CT': 'Dem', 'DE': 'Dem', 'FL': 'Rep', 'GA': 'Rep', 'HI': 'Dem', 'ID': 'Rep', 'IL': 'Dem', 'IN': 'Rep', 'IA': 'Rep', 'KS': 'Div', 'KY': 'Div', 'LA': 'Div', 'ME': 'Dem', 'MD': 'Div', 'MA': 'Div', 'MI': 'Div', 'MN': 'Div', 'MS': 'Rep', 'MO': 'Rep', 'MT': 'Div', 'NE': 'Rep', 'NV': 'Dem', 'NH': 'Div', 'NJ': 'Dem', 'NM': 'Dem', 'NY': 'Dem', 'NC': 'Div', 'ND': 'Rep', 'OH': 'Rep', 'OK': 'Rep', 'OR': 'Dem', 'PA': 'Div', 'RI': 'Dem', 'SC': 'Rep', 'SD': 'Rep', 'TN': 'Rep', 'TX': 'Rep', 'UT': 'Rep', 'VT': 'Div', 'VA': 'Dem', 'WA': 'Dem', 'WV': 'Rep', 'WI': 'Div', 'WY': 'Rep'}

In [319]:
covid_dates_cleaned['political_control_state'] = covid_dates_cleaned['state'].map(political_control_state)
covid_dates_cleaned.head()

,ranking,state,zip,ivy,institution_type,decision_type,days_after_first,political_control_state
0,0,NJ,8544,True,Private,online,8.0,Dem
1,0,NY,10027,True,Private,online,6.0,Dem
2,0,MA,2138,True,Private,online,4.0,Div
3,0,MA,2139,False,Private,online,4.0,Div
4,0,CT,6520,True,Private,online,5.0,Dem


### Preprocessing

Next, I'll turn state, zip, ivy, and institution type into categorical variables. To do this, we will use one-hot encoding. Note that my preprocessing is copied/based on [this sklearn course](https://inria.github.io/scikit-learn-mooc/python_scripts/03_categorical_pipeline_column_transformer.html).

Note there's only 23 states present in this

In [214]:
len(covid_dates_cleaned['state'].unique())

23

Don't use zip as we extracted data from it.

In [320]:
covid_dates_data = covid_dates_cleaned.drop(columns="zip").dropna() # drop rows if NaN
covid_dates_target = covid_dates_data["days_after_first"]
covid_dates_data.drop(columns="days_after_first", inplace=True)

In [321]:
covid_dates_data.dtypes

ranking                     int64
state                      object
ivy                          bool
institution_type           object
decision_type              object
political_control_state    object
dtype: object

In [322]:
from sklearn.compose import make_column_selector as selector # separate columns into numerical and categorical
numerical_columns_selector = selector(dtype_exclude=[object, bool])
categorical_columns_selector = selector(dtype_include=[object, bool])

numerical_columns = numerical_columns_selector(covid_dates_data)
categorical_columns = categorical_columns_selector(covid_dates_data)

In [323]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
categorical_preprocessor = OneHotEncoder(handle_unknown='ignore')
numerical_preprocessor = StandardScaler() # normalize data to make it easier for sklearn models to handle

In [324]:
from sklearn.compose import ColumnTransformer # splits the column, transforms each subset differently, then concatenates
preprocessor = ColumnTransformer([('one-hot-encoder', categorical_preprocessor, categorical_columns),
                                  ('standard_scalar', numerical_preprocessor, numerical_columns)])

In [325]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(covid_dates_data, covid_dates_target)

Now, create model. I'll use normal linear regression at first.

In [326]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(preprocessor, LinearRegression())
pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['state', 'ivy',
                                                   'institution_type',
                                                   'decision_type',
                                                   'political_control_state']),
                                                 ('standard_scalar',
                                                  StandardScaler(),
                                                  ['ranking'])])),
                ('linearregression', LinearRegression())])

In [327]:
pipe.score(X_test, y_test)

0.3152768081719818

Using my fairly random imputing method, I get a score of about -0.003, while with dropping the NaNs, I get a nicer result of 0.3.

Maybe should output a rank (i.e., which universities acted first)? Also, the ranking feature should probably be changed--not a linear relationship.

## Exploring a Different Task
*TODO* Can we create a classification model that tells us whether or not universities have a vaccine mandate? Is there even enough data to do this?